# Regression of Proposal Characteristics on CAR

In [4]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [5]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [6]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td

    * Create year variable
    gen year = year(day)

    * Create quarter variable
    gen quarter = qofd(day)
    format quarter %tq

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    egen token_year = group(token year)
    egen token_quarter = group(token quarter)

    * Define windows and specs
    local windows 2 5
    local vars    n_choices quadratic ranked_choice duration

    * Label variables
    label variable n_choices      "\$\textit{\# of Choices}_{i,t}\$"
    label variable quadratic      "\$\textit{Quadratic Voting}_{i,t}\$"
    label variable ranked_choice  "\$\textit{Ranked Choice Voting}_{i,t}\$"
    label variable duration        "\$\textit{Voting Duration}_{i,t}\$"

    eststo clear

    foreach w of local windows {
        preserve
        keep if index == `w'

        foreach v of local vars {
            * Run regression
            reghdfe car_`w' `v', absorb(token year) vce(cluster token)
            eststo car`w'_`v'

            * Tag FE rows (Y/N) for this spec
            estadd local fe_token "Y"
            estadd local fe_time  "Y"
        }

        restore
    }

    * Export a single big LaTeX table with 8 models (2 windows × 4 specs)
    esttab                                                          ///
        car2_n_choices car2_quadratic car2_ranked_choice car2_duration ///
        car5_n_choices car5_quadratic car5_ranked_choice car5_duration ///
        using "$TABLES/reg_car_proposals_`stage'.tex", replace          ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon        ///
        nonotes booktabs nomtitles                                 ///
        b(%9.3f) se(%9.2f)                                         ///
        mgroups("CAR[-2,2]" "CAR[-5,5]",                           ///
                span                                               ///
                pattern(1 0 0 1 0 0)                           ///
                prefix(\multicolumn{@span}{c}{)                    ///
                suffix(})                                          ///
                erepeat(\cmidrule(lr){@span}) )                    ///
        substitute("\_" "_")                                       ///
        stats(fe_token fe_time N r2_a,                             ///      
              fmt(0 0 %9.0fc %9.3f) labels("Token FE" "Year FE" "Observations" "Adjusted R²"))
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5. 
.     * Create year variable
.     gen year = year(day)
  6. 
.     * Create quarter variable
.     gen quarter = qofd(day)
  7.     format quarter %tq
  8. 
.     * Ensure index is numeric (safeguard if it came in as string)
.     capture confirm numeric variable index
  9.     if _rc {
 10.         destring index, replace ignore("[]")
 11.     }
 12. 
.     * Encode categorical variables
.     encode gecko_id, gen(token)
 13.     encode space,    gen(dao)
 14. 
.     egen token_year = group(token year)
 15.     egen token_quarter = group(token quarter)
 16. 
.     * Define windows and specs
.     local windows 2 5
 17.     local vars    n_choices quadratic ranked_choice duration
 18. 
.     * Label variables
.     label variable n_choices      "\$\textit{\# of Choices}_{i,t}